# Zenodo API Example

This notebook demonstrates how to query the Zenodo REST API. First of all, we create a couple of Python helper functions and classes to make it easier to query Zenodo. Don't worry too much about the details:

In [70]:
import requests

def raise_on_error(res):
    """Helper to check response for errors."""
    if res.status_code != 200:
        data = res.json()
        raise Exception('[{status}] {message}'.format(**data))

        
class ResultWrapper(object):
    """Helper to work with search results"""
    def __init__(self, session, response):
        raise_on_error(response)
        self._session = session
        self.response = response
        
    def has_json(self):
        """Check if content type is JSON."""
        return self.response.headers['Content-Type'] == 'application/json'
            
    @property
    def data(self):
        """Get JSON data."""
        return self.response.json() if self.has_json() else {}
    
    @property
    def aggregations(self):
        """Get aggregations."""
        return self.response.json()['aggregations']  if self.has_json() else {}
    
    @property
    def total(self):
        """Get total number of hits."""
        return self.response.json()['hits']['total'] if self.has_json() else None
        
    @property
    def pages(self):
        """Helper to fetch all result pages."""
        yield self.response
        next_url = self.response.links.get('next', {}).get('url')
        while next_url:
            res_page = self._session.get(next_url)
            raise_on_error(res_page)
            yield res_page
            next_url = res_page.links.get('next', {}).get('url')
    
    @property
    def hits(self):
        """Helper to iterate over each hit."""
        if not self.has_json():
            return None
        for res in self.pages:
            for h in res.json()['hits']['hits']:
                yield h
                
class ZenodoClient(object):
    """Simple Zenodo API Client"""

    def __init__(self, accept=None, token=None):
        self._accept = accept
        self._token = token
        self._session = None
        self._endpoint = 'https://www.zenodo.org/api/'
        self._endpoint_search = '{}records/'.format(self._endpoint)
        self._endpoint_communities = '{}communities/'.format(self._endpoint)
        self._endpoint_styles = '{}csl/styles'.format(self._endpoint)
        
    @property
    def session(self):
        """Create a session for making HTTP requests to the API."""
        if self._session is None:
            self._session = requests.Session()    
            # Construct headers
            headers = {
                'Accept': self._accept or 'application/vnd.zenodo.v1+json',
                'Accept-Charset': 'utf-8',
            }
            if self._token:
                headers['Authorization'] = 'Bearer {}'.format(self._token)            
            self._session.headers.update(headers)
        return self._session

    def search(self, query=None, size=None, sort=None, **filters):
        """Search Zenodo"""
        params = {'q': query or '', 'sort': sort or 'bestmatch', 'size': size or 100}
        if filters:
            for f, vals in filters.items():
                params[f] = vals
        return ResultWrapper(self.session, self.session.get(self._endpoint_search, params=params))
    
    def communities(self, query=None, size=None, sort=None, **filters):
        """Search Communities on Zenodo"""
        params = {'q': query or '', 'size': size or 1000}
        return ResultWrapper(self.session, self.session.get(self._endpoint_communities, params=params))
    
    def record(self, record_id, accept=None, **params):
        """Retrieve a single record."""
        url = '{base}{recid}'.format(base=self._endpoint_search, recid=record_id)
        params = params or {}
        headers = self.session.headers
        if accept:
            headers['Accept'] = accept
            
        res = self.session.get(url, headers=headers, params=params)
        raise_on_error(res)
        return res
    
    def csl_styles(self):
        res = self.session.get(self._endpoint_styles)
        raise_on_error(res)
        return res
        

## Initialize the API client
Using our just created helper classes we can now create a Zenodo API client:

In [71]:
api = ZenodoClient()

### Search
Using the API client we can now execute queries against Zenodo search API.


In [72]:
query = '+type:software +keywords:(marine ocean sea fish aqua)'
result = api.search(query)
result.total

38

Notice the query string. Here is a couple of points:

* **Field search**: We are now searching on two specific fields - ``type`` and ``keywords``.
* **Required fields**: The small ``+`` in front the field indicate the field must be present (i.e. both fields are required sincen they have a plus in the front).
* **Field multi-term**: On ``keywords`` we search for ``marine`` or ``ocean`` or ....

Now let's inspect the results:

In [30]:
for record in result.hits:
    print('{}: {}'.format(record['doi'], record['metadata']['title']))

10.5281/zenodo.161319: The Finite-volumE Sea ice-Ocean Model (FESOM2)
10.5281/zenodo.61540: OHI Canada survey results code: Final Publication Version
10.5281/zenodo.61716: OHI Canada survey results code: Final Publication Version
10.5281/zenodo.1157228: Finite Element Sea-ice/ice-shelf Ocean Model (FESOM) 1.4
10.5281/zenodo.1157230: MetROMS-iceshelf
10.5281/zenodo.32741: Data Processing for a Small-Scale Long-Term Coastal Ocean Observing System Near Mobile Bay, Alabama: A Geoscience Papers of the Future (GPF) Software Set
10.5281/zenodo.574902: Analysis and Figure Creation From: "Incorporating public priorities in the Ocean Health Index: Canada as a case study"
10.5281/zenodo.574901: Data Wrangling From: "Incorporating public priorities in the Ocean Health Index: Canada as a case study"
10.5281/zenodo.1116851: The Finite-Element Sea ice-Ocean Model (FESOM)
10.5281/zenodo.247741: UNC-CFD/somar: SOMAR first release
10.5281/zenodo.15507: LocalizeSL: Offline sea-level localization code for

## Export a bibliograph
Zenodo supports several other output metadata formats other than JSON. For search results Zenodo can produce the following metadata formats:

* BibTex: ``application/x-bibtex``
* DublinCore: ``application/x-dc+xml``
* DataCite: ``application/x-datacite+xml``
* MARC21: ``application/marcxml+xml``

For individual records the following additional formats can be produced:

* JSON-LD: ``application/ld+json``
* Citation Style Language (CSL): ``application/vnd.citationstyles.csl+json``
* Citation text: ``text/x-bibliography``

In the first example, we will download a BibTeX bibliography of all records in above search query and write it to a file:

In [31]:
# Create a API client (specifying the desired output format)
api = ZenodoClient(accept='application/x-bibtex')

# Query for records belonging to the Astronomy Thesis Collection.
result = api.search(query)

with open('bibliography.bib', 'wb') as fp:
    # Iterate over all the results and write them to the file.
    for page in result.pages:
        fp.write(page.content)

In the second example we will generate a citation string for all the records:

In [22]:
api = ZenodoClient()
result = api.search(query)
for h in result.hits:
    res = api.record(h['id'], accept='text/x-bibliography', style='apa')
    print(res.text)

Sergey Danilov, Dmitry Sidorenko, Qiang Wang, & Thomas Jung. (2016, October 17). The Finite-volumE Sea ice-Ocean Model (FESOM2). Zenodo. http://doi.org/10.5281/zenodo.161319
Remi Daigle, ., Wolfgang Haider, ., Sergio Fernández-Lozada, ., Kimberly Irwin, ., Philippe Archambault, ., & Isabelle M. Côté, . (2016, September 5). OHI Canada survey results code: Final Publication Version (Version v1.0). Zenodo. http://doi.org/10.5281/zenodo.61540
Daigle, R., Haider, W., Fernández-Lozada, S., Irwin, K., Archambault, P., & Côté, I. M. (2016, September 7). OHI Canada survey results code: Final Publication Version. Zenodo. http://doi.org/10.5281/zenodo.61716
Naughten, Kaitlin A., Meissner, Katrin J., Galton-Fenzi, Benjamin K., England, Matthew H., Timmermann, Ralph, Hellmer, Hartmut H., … Debernard, Jens B. (2018, January 23). Finite Element Sea-ice/ice-shelf Ocean Model (FESOM) 1.4. Zenodo. http://doi.org/10.5281/zenodo.1157228
Naughten, Kaitlin A., Meissner, Katrin J., Galton-Fenzi, Benjamin K.,

# Aggregations
Want to know about types, open/closed access, file types and keywords?

In [23]:
api = ZenodoClient()
result = api.search(query)
for b in result.aggregations['type']['buckets']:
    print('{key}: {doc_count}'.format(**b))


software: 38


In [24]:
for b in result.aggregations['access_right']['buckets']:
    print('{key}: {doc_count}'.format(**b))

open: 38


In [25]:
for b in result.aggregations['keywords']['buckets']:
    print('{key}: {doc_count}'.format(**b))

ocean: 21
sea: 11
marine: 9
model: 9
ice: 6
modeling: 5
modelling: 5
python: 5
data: 4
climate: 3


In [26]:
for b in result.aggregations['file_type']['buckets']:
    print('{key}: {doc_count}'.format(**b))

zip: 31
gz: 5
sce: 1
tar: 1


# Communities

In [65]:
api = ZenodoClient(accept='application/json')
keywords = ['marine', 'ocean', 'fish', 'aqua' ]
communities = {}

for k in keywords:
    result = api.communities(k)
    for c in result.hits:
        communities[c['id']] = c

In [66]:
len(communities)

44

In [68]:
for c, data in sorted(communities.items()):
    print('https://www.zenodo.org/communities/{}/'.format(c))

https://www.zenodo.org/communities/2231-0606/
https://www.zenodo.org/communities/2231-3443/
https://www.zenodo.org/communities/2231-4776/
https://www.zenodo.org/communities/adriplan/
https://www.zenodo.org/communities/aquarius/
https://www.zenodo.org/communities/aquatrace/
https://www.zenodo.org/communities/assisibf/
https://www.zenodo.org/communities/atlas/
https://www.zenodo.org/communities/bermudabream/
https://www.zenodo.org/communities/bigdataocean/
https://www.zenodo.org/communities/biogeochem/
https://www.zenodo.org/communities/blue-actionh2020/
https://www.zenodo.org/communities/brcorp1/
https://www.zenodo.org/communities/calcifierraman/
https://www.zenodo.org/communities/columbus/
https://www.zenodo.org/communities/devotes-project/
https://www.zenodo.org/communities/discardless/
https://www.zenodo.org/communities/eawag/
https://www.zenodo.org/communities/ecc2014/
https://www.zenodo.org/communities/euro-basin/
https://www.zenodo.org/communities/facts/
https://www.zenodo.org/com